<a href="https://colab.research.google.com/github/harshhrawte/PrepNexus/blob/main/roberta_model_trainmodelpySC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets PyPDF2 scikit-learn joblib -q

import os
import pandas as pd
import torch
import joblib
import PyPDF2
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving grand_99.csv to grand_99.csv


In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


os.environ["WANDB_DISABLED"] = "true"


df = pd.read_csv("grand_99.csv")
df.fillna('', inplace=True)
df['contact info'] = df['contact info'].astype(str)

def preprocess_data(df):
    df['text'] = (
        "Skills: " + df['skills'].astype(str) + " | " +
        "Experience: " + df['experience'].astype(str) + " | " +
        "Domain: " + df['domain'].astype(str) + " | " +
        "Education: " + df['education'].astype(str) + " | " +
        "Certifications: " + df['certifications'].astype(str) + " | " +
        "Projects: " + df['projects'].astype(str)
    )
    return df[['text', 'predicted role']]

df = preprocess_data(df)


labels = {role: i for i, role in enumerate(df['predicted role'].unique())}
df['label'] = df['predicted role'].map(labels)

#imbalance
df = df.groupby('predicted role').apply(lambda x: x.sample(min(len(x), 300), random_state=42)).reset_index(drop=True)


Using device: cuda


<ipython-input-5-d5ea3a166d0e>:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)
<ipython-input-5-d5ea3a166d0e>:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('predicted role').apply(lambda x: x.sample(min(len(x), 300), random_state=42)).reset_index(drop=True)


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, stratify=df['label'], random_state=42
)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=256)

train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels}).map(tokenize_function, batched=True)
val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels}).map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/804 [00:00<?, ? examples/s]

Map:   0%|          | 0/201 [00:00<?, ? examples/s]

In [ ]:
#abhi shuru hua classification fine tuning
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(labels))
model.gradient_checkpointing_enable()
model.to(device)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    do_eval=True,  # Enables evaluation (older equivalent of evaluation_strategy)
    save_steps=500,  # Save every 500 steps (approximation)
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    logging_steps=100,  # Optional: log every 100 steps
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
100,4.150700
200,3.570400
300,3.082200
400,2.876700


TrainOutput(global_step=404, training_loss=3.414066428005105, metrics={'train_runtime': 76.4362, 'train_samples_per_second': 42.074, 'train_steps_per_second': 5.285, 'total_flos': 423344686473216.0, 'train_loss': 3.414066428005105, 'epoch': 4.0})

In [ ]:
model_dir = "./roberta_resume"
os.makedirs(model_dir, exist_ok=True)

model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)
joblib.dump(labels, os.path.join(model_dir, "label_mapping.pkl"))

print("✅ Model & label mapping saved!")


✅ Model & label mapping saved!


In [ ]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        return " ".join([page.extract_text() for page in reader.pages if page.extract_text()])

def predict_role(pdf_path):
    model = RobertaForSequenceClassification.from_pretrained(model_dir)
    tokenizer = RobertaTokenizer.from_pretrained(model_dir)
    labels = joblib.load(os.path.join(model_dir, "label_mapping.pkl"))
    model.to(device)
    model.eval()

    resume_text = extract_text_from_pdf(pdf_path)
    inputs = tokenizer(resume_text, return_tensors="pt", truncation=True, padding='max_length', max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()

    return list(labels.keys())[prediction]


In [ ]:
uploaded = files.upload()

pdf_path = next(iter(uploaded))
predicted_role = predict_role(pdf_path)
print(f"🧠 Predicted Role: {predicted_role}")


Saving Harsh_Rawte_Resume.pdf to Harsh_Rawte_Resume.pdf
🧠 Predicted Role: Full-Stack Developer, Software Engineer


In [ ]:
!zip -r roberta_resume.zip /content/roberta_resume


  adding: content/roberta_resume/ (stored 0%)
  adding: content/roberta_resume/label_mapping.pkl (deflated 58%)
  adding: content/roberta_resume/config.json (deflated 72%)
  adding: content/roberta_resume/merges.txt (deflated 53%)
  adding: content/roberta_resume/vocab.json (deflated 68%)
  adding: content/roberta_resume/special_tokens_map.json (deflated 84%)
  adding: content/roberta_resume/tokenizer_config.json (deflated 76%)
  adding: content/roberta_resume/model.safetensors (deflated 17%)


In [ ]:
from google.colab import files
files.download("roberta_resume.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>